In [11]:
import socket
import os
from _thread import *
from random import randrange, getrandbits
from math import gcd
import json
import time

### Generating big prime numbers using Miller Rabin Test
<a href='https://medium.com/@ntnprdhmm/how-to-generate-big-prime-numbers-miller-rabin-49e6e6af32fb'>Source</a>

In [12]:
def is_prime(n, k=128):
    """ Test if a number is prime
        Args:
            n -- int -- the number to test
            k -- int -- the number of tests to do
        return True if n is prime
    """
    # Test if n is not even.
    # But care, 2 is prime !
    if n == 2 or n == 3:
        return True
    if n <= 1 or n % 2 == 0:
        return False
    # find r and s
    s = 0
    r = n - 1
    while r & 1 == 0:
        s += 1
        r //= 2
    # do k tests
    for _ in range(k):
        a = randrange(2, n - 1)
        x = pow(a, r, n)
        if x != 1 and x != n - 1:
            j = 1
            while j < s and x != n - 1:
                x = pow(x, 2, n)
                if x == 1:
                    return False
                j += 1
            if x != n - 1:
                return False
    return True

def generate_prime_candidate(length):
    """ Generate an odd integer randomly
        Args:
            length -- int -- the length of the number to generate, in bits
        return a integer
    """
    # generate random bits
    p = getrandbits(length)
    # apply a mask to set MSB and LSB to 1
    p |= (1 << length - 1) | 1
    return p
def generate_prime_number(length=1024):
    """ Generate a prime
        Args:
            length -- int -- length of the prime to generate, in          bits
        return a prime
    """
    p = 4
    # keep generating while the primality test fail
    while not is_prime(p, 128):
        p = generate_prime_candidate(length)
    return p


### Key Generation

In [13]:
def generate_public_key(phi):
    e = phi-1
    while True:
        if gcd(e, phi) == 1:
            return e
        e -= 1

In [14]:
def generate_key():
    p = generate_prime_number()
    q = generate_prime_number()
    n = p * q
    phi = (p-1) * (q-1)
    e = generate_public_key(phi)
    d = pow(e, -1, phi)
    return (e, n), (d, n)

### Encryption

In [15]:
def encrypt(pk, plaintext):
    # Unpack the key into it's components
    key, n = pk
    # Convert each letter in the plaintext to numbers based on the character using a^b mod m
    cipher = [pow(ord(char), key, n) for char in plaintext]
    # Return the array of bytes
    return cipher

### Decryption

In [16]:
def decrypt(pk, ciphertext):
    # Unpack the key into its components
    key, n = pk
    # Generate the plaintext based on the ciphertext and key using a^b mod m
    plain = [chr(pow(char, key, n)) for char in ciphertext]
    # Return the array of bytes as a string
    return ''.join(plain)

In [17]:
def generate_id():
    return os.urandom(16).hex()

def get_timestamp():
    return int(time.time())

def get_nonce():
    # generate a random 64-bit number between 2^32 and 2^64-1
    return randrange(2**64)

def get_nonce_acknowledgment(nonce_received):
    # XORing with a constant value twice gives back the original value
    return nonce_received ^ 0xFFFF 

def get_validity_timestamp():
    return get_timestamp() + 5*60*1000

def check_validity_message(valid_till):
    return valid_till > get_timestamp()


In [18]:
def add_public_key_to_ring(public_key, public_key_ring, client_id):
    public_key_ring[client_id] = public_key

def get_public_key_from_ring(public_key_ring, client_id):
    return public_key_ring[client_id]

In [21]:
ClientMultiSocket = socket.socket()
host = '127.0.0.1'
port = 8080
public_key, private_key = generate_key()
id = generate_id()
public_key_ring = {}
try:
    ClientMultiSocket.connect((host, port))
    
except socket.error as e:
    print(str(e))

## message structure
# {
#     "type":'handshake', # type of message
#     'id':'', # unique id of the sender
#     'public_key': '', # public key of the sender
#     'timestamp': '', # timestamp of the message
#     'nonce': '', # nonce of the message
#     'valid_till': '', # validity timestamp of the message
#     'acknowledgement_nonce': '' # f(previous_message_nonce) XORed with a constant value
# }

## first message in the channel 
latest_message_nonce = get_nonce()
client_handshake_message = {
    "type":'handshake',
    "id": id,
    "public_key": public_key,
    'timestamp': get_timestamp(),
    'nonce':latest_message_nonce,
    'valid_till': get_validity_timestamp()
    # this message doesn't have an acknowledgement nonce because it is the first message
}

client_handshake_message_str = json.dumps(client_handshake_message)
ClientMultiSocket.send(str.encode(client_handshake_message_str))

server_handshake_message_str = ClientMultiSocket.recv(2048).decode('utf-8')
server_handshake_message = json.loads(server_handshake_message_str)

if not check_validity_message(server_handshake_message['valid_till']):
    print('Invalid message, message expired')
else:
    print(server_handshake_message)
    acknowledgement_nonce = server_handshake_message['acknowledgement_nonce']
    if latest_message_nonce != get_nonce_acknowledgment(acknowledgement_nonce):
        print('The server did not acknowledge the handshake message, nonce mismatch')
    else:
        print('Handshake successful')
        add_public_key_to_ring(server_handshake_message["public_key"], public_key_ring, server_handshake_message["id"])




ClientMultiSocket.close()
print(public_key_ring)

{'type': 'handshake', 'id': 'public key distribution authority', 'public_key': [22905204799085418614470651110036475527822313997049748720900317422023991344291981446708421163337669507222155406787032223360688879819978438866594098252295238810300972921745419923492508500174408006902452628373624618817305961220456986451316056497296337128314355613377832898729061765272289198605223986644218380991668972402454641735641918263550579911373639907784188830292709760310291834649581724823320363786652818245266735588622423996574767514960735107200016924248922055077159355547348393664298428388736849304738499078951690308474158824474199546165634722566513621219749387300965030838996937653937551918630833547613860656903, 2290520479908541861447065111003647552782231399704974872090031742202399134429198144670842116333766950722215540678703222336068887981997843886659409825229523881030097292174541992349250850017440800690245262837362461881730596122045698645131605649729633712831435561337783289872906176527228919860522398664421